In [30]:
# check entailment

from sympy import symbols, And, Not, Or, Implies, Equivalent
from sympy.logic.boolalg import truth_table
from sympy.logic.inference import satisfiable

A = symbols('A')
B = symbols('B')
C = symbols('C')
D = symbols('D')

statement_A = Implies(A, ~B)
statement_B = Implies(~A, And(B, C))
statement_C = Implies(Or(And(B, C),And(~B, ~C)),D)
statement_D = Implies(D, A)

WFF1 = And(Equivalent(A,Implies(B,~C)),Equivalent(B,Or(And(~A,C),And(A,~C))),Equivalent(C,~A))
WFF2 = Implies(A, B)

KB = And(statement_A, statement_B, statement_C, statement_D)
symbols_list = [A, B, C]
# method 1: check entailment with truth table
for assignment, value in truth_table(WFF1, symbols_list):
    print(f"Assignment: {assignment}, Value: {value}")
# print("\n")
# for assignment, value in truth_table(WFF2, symbols_list):
#     print(f"Assignment: {assignment}, Value: {value}")
    

# method 2: check KB & ~query is contradiction
def contradiction_check(KB, query):
    return not satisfiable(And(KB, Not(query)))

# print("KB entails A?", contradiction_check(KB, A))
# print("KB entails ~A?", contradiction_check(KB, ~A))
# print("KB entails B?", contradiction_check(KB, B))
# print("KB entails ~B?", contradiction_check(KB, ~B))
# print("KB entails B?", contradiction_check(KB, C))
# print("KB entails ~B?", contradiction_check(KB, ~C))

print(list(satisfiable(KB, all_models=True)))


Assignment: [0, 0, 0], Value: False
Assignment: [0, 0, 1], Value: False
Assignment: [0, 1, 0], Value: False
Assignment: [0, 1, 1], Value: True
Assignment: [1, 0, 0], Value: False
Assignment: [1, 0, 1], Value: False
Assignment: [1, 1, 0], Value: True
Assignment: [1, 1, 1], Value: False
[{C: True, D: True, A: True, B: False}, {C: True, A: True, D: False, B: False}, {D: True, A: True, C: False, B: False}]


In [15]:
# grid & neighbours
grid_size = 3
P = {}
W = {}
B = {}
S = {}

# creating symbols
for x in range(1, grid_size+1):
    for y in range(1, grid_size+1):
        P[(x, y)] = symbols(f'P_{x}_{y}')
        W[(x, y)] = symbols(f'W_{x}_{y}')
        B[(x, y)] = symbols(f'B_{x}_{y}')
        S[(x, y)] = symbols(f'S_{x}_{y}')

# --- generating rules ---

# retrieving neighbours   
def get_neighbours(x, y, grid_size):
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    neighbours = [(x + dx, y + dy) for dx, dy in directions]
    return [(nx, ny) for nx, ny in neighbours if 1 <= nx <= grid_size and 1 <= ny <= grid_size]

# implies check
def Pit_implies_Breeze(x, y, P, B, grid_size):
    # splat operator unpacks list to separate arguments
    adjacent_breeze = And(*[B[adj] for adj in get_neighbours(x, y, grid_size)])
    pit_implies_breeze = Implies(P[(x,y)], adjacent_breeze)
    return pit_implies_breeze

# check at least
def at_least_one_pit(P, grid_size):
    return Or(*[P[(x,y)] for x in range (1, grid_size+1) for y in range (1, grid_size+1)])

In [16]:
# --- combining all rules for all grids ---
rules = set()
for x in range(1, grid_size+1):
    for y in range(1, grid_size+1):
        pit_implies_breeze = Pit_implies_Breeze(x, y, P, B, grid_size)
        rules.add(pit_implies_breeze)
rules.add(at_least_one_pit(P, grid_size))
rules

{Implies(P_1_1, B_1_2 & B_2_1),
 Implies(P_1_2, B_1_1 & B_1_3 & B_2_2),
 Implies(P_1_3, B_1_2 & B_2_3),
 Implies(P_2_1, B_1_1 & B_2_2 & B_3_1),
 Implies(P_2_2, B_1_2 & B_2_1 & B_2_3 & B_3_2),
 Implies(P_2_3, B_1_3 & B_2_2 & B_3_3),
 Implies(P_3_1, B_2_1 & B_3_2),
 Implies(P_3_2, B_2_2 & B_3_1 & B_3_3),
 Implies(P_3_3, B_2_3 & B_3_2),
 P_1_1 | P_1_2 | P_1_3 | P_2_1 | P_2_2 | P_2_3 | P_3_1 | P_3_2 | P_3_3}

In [17]:
KB = rules.copy()
current_loc = (1,1)

# update KB with new percepts/initial cond
KB.add(~B[current_loc])
KB.add(~S[current_loc])
KB.add(~P[current_loc])
KB.add(~W[current_loc])
KB_AND = And(*KB)

# check query with infer
def Infer(KB_AND, query):
    entails = not satisfiable(And(KB_AND & ~query))
    if entails:
       print(query, "is definitely true.")
    else:
        entails_not = not satisfiable(KB_AND & query)
        if entails_not:
            print(query, "is definitely false.")
        else:
            print(query,"is uncertain.")

# inference
query = W[(2,2)]
Infer(KB_AND, query)
query = P[(2,2)]
Infer(KB_AND, query)


W_2_2 is uncertain.
P_2_2 is uncertain.


In [18]:
# inference
current_loc = (1,1)
neighbours = get_neighbours(current_loc[0], current_loc[1], grid_size)
for neighbour in neighbours:
    Infer(KB_AND, W[neighbour])
    Infer(KB_AND, P[neighbour])

# update (with inference results)
KB.add(~P[(2,1)])
KB.add(~P[(1,2)])

# update KB (with new percepts)
current_loc = (2,1)
KB.add(B[current_loc])
KB.add(~S[current_loc])
KB_AND = And(*KB)

# inference
neighbours = get_neighbours(current_loc[0], current_loc[1], grid_size)
for neighbour in neighbours:
    Infer(KB_AND, P[neighbour])
    Infer(KB_AND, W[neighbour])

W_2_1 is uncertain.
P_2_1 is definitely false.
W_1_2 is uncertain.
P_1_2 is definitely false.
P_1_1 is definitely false.
W_1_1 is definitely false.
P_3_1 is uncertain.
W_3_1 is uncertain.
P_2_2 is uncertain.
W_2_2 is uncertain.
